### **Scraping EL PAIS**

### Dependencias iniciales

In [10]:
import time
import requests
from lxml import html
import json

### Configuración de las peticiones

In [5]:
URL_BASE = "https://elpais.com/buscador/Trump/{}"

MAX_PAGINAS = 7
DELAY = 1.2

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/141.0.0.0 Safari/537.36 OPR/125.0.0.0",
    "Referer": "https://elpais.com/mexico/",
    "Upgrade-Insecure-Requests": "1",
    "sec-ch-ua": '"Opera GX";v="125", "Not?A_Brand";v="8", "Chromium";v="141"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'
}

In [6]:
noticias = []

for pagina in range(1, MAX_PAGINAS + 1):

    try:
        url = URL_BASE.format(pagina)
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        tree = html.fromstring(response.text)

        articulos = tree.xpath('//section[@id="search-results"]//div[@id="results-container"]//article')
    
    except Exception as e:
        print(f"Error al intentar obtener la página {pagina}", e)
        continue

    for articulo in articulos:
        try:
            link = articulo.xpath('.//h2//a/@href')

            if not link:
                continue

            url_noticia = link[0]

            titulo = articulo.xpath('.//h2//a/text()')
            titulo = titulo[0].strip() if titulo else None

            autor = articulo.xpath('.//div[contains(@class,"c_a")]//a/text()')
            autor = autor[0].strip() if autor else None

            descripcion = articulo.xpath('.//p[contains(@class,"c_d")]/text()')
            descripcion = descripcion[0].strip() if descripcion else None

            fecha = articulo.xpath('.//div[contains(@class,"c_f")]/text()')
            fecha = fecha[0].strip() if fecha else None

            noticia = {
                "titulo": titulo,
                "url": url_noticia,
                "autor": autor,
                "fecha": fecha,
                "descripcion": descripcion,
            }

            noticias.append(noticia)
        except Exception as e:
            print("Error procesando el articulo: ", e)
            continue

In [9]:
print(f"Total de noticias obtenidas: {len(noticias)}")

if noticias:
    print("\nPrimera noticia obtenida:")
    print("Título:", noticias[0]["titulo"])
    print("Descripcion:", noticias[0]["descripcion"])
    print("Fecha:", noticias[0]["fecha"])
    print("URL:", noticias[0]["url"])

Total de noticias obtenidas: 97

Primera noticia obtenida:
Título: Ni apaciguar ni confiar en Trump
Descripcion: De la debilidad y el descrédito que el presidente de EE UU ha exhibido en Davos no cabe deducir una súbita fortaleza de los europeos
Fecha: 25/01/2026
URL: https://elpais.com/opinion/2026-01-25/ni-apaciguar-ni-confiar-en-trump.html


In [11]:
for i, noticia in enumerate(noticias):
    try:
        response = requests.get(noticia["url"], headers=HEADERS, timeout=10)
        response.raise_for_status()

        tree = html.fromstring(response.text)

        parrafos = tree.xpath(
            '//article[@id="main-content"]//div[contains(@class,"a_c")]//p'
        )

        contenido = [
            p.text_content().strip()
            for p in parrafos
            if p.text_content().strip()
        ]

        noticia["contenido"] = " ".join(contenido) if contenido else None

        time.sleep(1.2)

    except Exception as e:
        print(f"Fallo en la noticia {i}: {noticia['url']}")
        noticia["contenido"] = None

In [12]:
with open("elpais.json", "w", encoding="utf-8") as f:
    json.dump(noticias, f, ensure_ascii=False, indent=2)

print(f"Total de noticias: {len(noticias)}")

Total de noticias: 97
